### Setup

In [55]:
# import pandas
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf

# exchange codes http://finabase.blogspot.com/2014/09/interantional-stock-exchange-codes-for.html

path = './Compustat data4.csv.nosync.csv'
df = pd.read_csv(path, sep=',')

/Users/vini2001/miniforge3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10,12,16,26,30,945,946,948,949,950,955,969,970,975) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Configurations

In [56]:
year_start = 2015
year_end = 2016
min_tickers_to_select_row = 4000
only_tickers_with_fiscal_year_ending_at = 12

### Code


#### Remove some exchanges and drop rows without market value

In [57]:
print(len(df))
# drop all df rows where exchg is [0, 1, 2, 19]
df = df[(df['exchg'] != 0) & (df['exchg'] != 1) & (df['exchg'] != 2)]

# # OTCs - it might be better to keep them, because if we exclude them, we will be selecting them if they were OTC at year_start but are not anymore, since compustat updates the exchg for each year with the current value
#                                   So if we use the old values not listed as OTC only, we'll be selecting only the OTCs with great performance, which would be misleading
df = df[(df['exchg'] != 13) & (df['exchg'] != 19)]

# canada
df = df[(df['exchg'] != 7) & (df['exchg'] != 8) & (df['exchg'] != 9)]

# drop rows without market value
df = df[df['mkvalt'].notna()]
print(len(df))

# tickers_to_exclude = ['NE', 'EVTL', 'VAL', 'CHK', 'GPOR', 'FYBR', 'CBL', 'TUEM', 'DTLA.P']
# df = df[~df['tic'].isin(tickers_to_exclude)]
# print(len(df))

243677
112925


#### Aux functions

In [74]:

path_items_descriptions = './items_descriptions.csv'
df_items_descriptions = pd.read_csv(path_items_descriptions, sep=',')
def get_description(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['ItemName'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['ItemDesc']
    return vals.values[0] if len(vals) > 0 else 'Not found'

def get_file_tag(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['FileTag'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['FileTag']
    return vals.values[0] if len(vals) > 0 else 'Not found'

def val_for_tic_at_year(tic, year, month=None):
    msft = yf.Ticker(tic)
    for i in range(0, 5):
        try:
            hist = msft.history(period="max")
            hist.to_csv('hist.csv')
            hist = pd.read_csv('hist.csv')
            break
        except:
            print(f'error getting hist for {tic} {year} {month}')
            continue
        
    dft = df[(df['tic'] == tic) & (df['fyear'] == year)]
    date = dft['datadate'].values[0]
    if month != None : 
        month += 1
        monthN = month if month <= 12 else month - 12
        date = f'{year if month <= 12 else year+1}{monthN if monthN > 9 else f"0{monthN}" }15'

    try:
        date = pd.to_datetime(date, format='%Y%m%d')
    except:
        date = pd.to_datetime(date, format='%Y-%m-%d')
    close = None
    if not 'Date' in hist: return None
    for i in range(0, 5):
        dateStr = date.strftime('%Y-%m-%d')
        valsDay = hist[hist['Date'] == dateStr]
        if len(valsDay) > 0:
            close = valsDay['Close'].values[0]
            break
        else:
            date = date + pd.DateOffset(days=1)
    return close

def market_caps_for_year(year):
    dfyear = df[df['fyear'] == year]
    # return df with mkvalt and tic
    dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']
    dfyear = dfyear[['tic', 'mkvalt', 'price', 'ajex', 'prcc_f']]
    #drop nans
    dfyear = dfyear.dropna()
    return dfyear



#### Most frequent columns

In [59]:
columns = df.columns.tolist()
print(columns)

df.head()


tickers = df['tic']
# get set of unique tickers
tickers = set(tickers)
print(f'{len(tickers)} tickers')

path = 'tickers.csv'
df_tickers = pd.DataFrame(tickers)
df_tickers.to_csv(path, index=False)

df['fyear']

['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'cusip', 'conm', 'acctchg', 'acctstd', 'acqmeth', 'adrr', 'ajex', 'ajp', 'bspr', 'compst', 'curcd', 'curncd', 'currtr', 'curuscn', 'final', 'fyr', 'ismod', 'ltcm', 'ogm', 'pddur', 'scf', 'src', 'stalt', 'udpl', 'upd', 'apdedate', 'fdate', 'pdate', 'acchg', 'acco', 'accrt', 'acdo', 'aco', 'acodo', 'acominc', 'acox', 'acoxar', 'acqao', 'acqcshi', 'acqgdwl', 'acqic', 'acqintan', 'acqinvt', 'acqlntal', 'acqniintc', 'acqppe', 'acqsc', 'act', 'adpac', 'aedi', 'afudcc', 'afudci', 'aldo', 'am', 'amc', 'amdc', 'amgw', 'ano', 'ao', 'aocidergl', 'aociother', 'aocipen', 'aocisecgl', 'aodo', 'aol2', 'aoloch', 'aox', 'ap', 'apalch', 'apb', 'apc', 'apofs', 'aqa', 'aqc', 'aqd', 'aqeps', 'aqi', 'aqp', 'aqpl1', 'aqs', 'arb', 'arc', 'arce', 'arced', 'arceeps', 'artfs', 'at', 'aul3', 'autxr', 'balr', 'banlr', 'bast', 'bastr', 'batr', 'bcef', 'bclr', 'bcltbl', 'bcnlr', 'bcrbl', 'bct', 'bctbl', 'bctr', 'billexce', 'bkvlps', 'bltb

0         1999
1         2000
2         2001
3         2002
4         2003
          ... 
243645    2022
243649    2022
243654    2021
243658    2021
243659    2022
Name: fyear, Length: 112925, dtype: int64

In [60]:

def most_frequent_columns(df):
    map_count = {}
    for c in columns:
        map_count[c] = df[c].describe()['count']

    # sort by count
    map_count = {k: v for k, v in sorted(map_count.items(), key=lambda item: item[1], reverse=True)}

    columns_data = []
    for k, v in map_count.items():
        columns_data.append([k, v, get_description(k), get_file_tag(k)])

    df_columns = pd.DataFrame(columns_data, columns=['Column', 'Count', 'Description', 'FileTag'])
    return df_columns

df_columns = most_frequent_columns(df)

path = './columns.csv'
df_columns.to_csv(path, index=False)
df_columns.head()

,Column,Count,Description,FileTag
0,gvkey,112925.0,Global Company Key - Key Selection,Multiple Files
1,datadate,112925.0,Data Date,Multiple Files
2,fyear,112925.0,Data Year - Fiscal,DES_ANN
3,indfmt,112925.0,Not found,Not found
4,consol,112925.0,Level of Consolidation,CCM_FILEDATE


#### Select rows for year_start

In [61]:
rows_start = df[df['fyear'] == year_start]
rows_start = rows_start.reset_index(drop=True)

# remove this line if datadate already comes as int
# rows_start['datadate'] = rows_start['datadate'].str.replace('-', '').astype(int)

rows_start = rows_start[rows_start['datadate'] <= int(f'{year_start if only_tickers_with_fiscal_year_ending_at <= 12 else year_start+1}{only_tickers_with_fiscal_year_ending_at}31')]

tickers_start = rows_start['tic']
tickers_start = set(tickers_start)
print(f'{len(tickers_start)} tickers in {year_start}')

df_columns_start = most_frequent_columns(rows_start)
path = f'./columns_{year_start}.csv'
df_columns_start.to_csv(path, index=False)
df_columns_start

3752 tickers in 2015


,Column,Count,Description,FileTag
0,gvkey,4400.0,Global Company Key - Key Selection,Multiple Files
1,datadate,4400.0,Data Date,Multiple Files
2,fyear,4400.0,Data Year - Fiscal,DES_ANN
3,indfmt,4400.0,Not found,Not found
4,consol,4400.0,Level of Consolidation,CCM_FILEDATE
...,...,...,...,...
976,xdepl,0.0,Depletion Expense (Schedule VI),IS_ANN_INDL
977,xdp,0.0,Depreciation Expense (Schedule VI),IS_ANN_INDL
978,add3,0.0,Address Line 3,COMPANY
979,add4,0.0,Address Line 4,COMPANY


In [68]:
min_tickers_to_select_row = (4500/4740) * len(rows_start['tic'])
print(min_tickers_to_select_row)
# select columns with count > min_count
def select_columns(df, min_count, file_tag):
    vals = df[df_columns_start['FileTag'] == file_tag]
    vals = vals[vals['Count'] > min_count]['Column']
    return vals.values.tolist()

vals_bal_ann_indl = select_columns(df_columns_start, min_tickers_to_select_row, 'BAL_ANN_INDL')
vals_is_ann_indl = select_columns(df_columns_start, min_tickers_to_select_row, 'IS_ANN_INDL')
val_cf_ann = select_columns(df_columns_start, min_tickers_to_select_row, 'CF_ANN')

print(f'{len(vals_bal_ann_indl)} columns in BAL_ANN_INDL')
print(f'{len(vals_is_ann_indl)} columns in IS_ANN_INDL')
print(f'{len(val_cf_ann)} columns in CF_ANN')

# select all rows in rows_start with columns of vals_bal_ann_indl and vals_is_ann_indl different than NaN
rows_start_bal_and_is = rows_start[['tic', 'fyear'] + vals_bal_ann_indl + vals_is_ann_indl + val_cf_ann + ['mkvalt', 'prcc_f']]
rows_start_bal_and_is = rows_start_bal_and_is.dropna()
rows_start_bal_and_is = rows_start_bal_and_is.reset_index(drop=True)
# print(len(rows_start_bal_and_is))
print(f'{len(rows_start_bal_and_is)} rows with all columns in {year_start}')

# create csv file
path = f'./rows_{year_start}_bal_ann_indl.csv'
rows_start_bal_and_is.to_csv(path, index=False)

tickers = rows_start_bal_and_is['tic']

data = rows_start_bal_and_is
data

4177.215189873417
21 columns in BAL_ANN_INDL
9 columns in IS_ANN_INDL
0 columns in CF_ANN
3588 rows with all columns in 2015


,tic,fyear,ao,at,icapt,lse,pstkr,tstkp,ceq,dltt,...,do,dvp,ib,revt,xido,dvc,dvt,ibmii,mkvalt,prcc_f
0,AAL,2015,4580.000,48415.000,23965.000,48415.000,0.0,0.0,5635.000,18330.000,...,0.000,0.0,7610.000,40990.000,0.000,278.000,278.000,7610.000,26452.7417,42.35
1,CECO,2015,3.925,598.819,402.855,598.819,0.0,0.0,239.153,157.834,...,0.000,0.0,-5.602,367.422,0.000,7.977,7.977,-5.734,260.4902,7.68
2,PRG,2015,63.138,2658.875,1819.890,2658.875,0.0,0.0,1366.618,453.272,...,0.000,0.0,135.709,3179.756,0.000,6.822,6.822,135.709,1625.5140,22.39
3,ABT,2015,2121.000,41247.000,27197.000,41247.000,0.0,0.0,21211.000,5871.000,...,1817.000,0.0,2606.000,20405.000,1817.000,1464.000,1464.000,2623.000,66137.3852,44.91
4,ACU,2015,1.029,81.421,69.097,81.421,0.0,0.0,43.184,25.913,...,0.000,0.0,4.794,109.812,0.000,1.237,1.237,4.794,58.2552,17.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3583,FRO,2015,3.603,2886.654,2641.777,2886.654,0.0,0.0,1446.282,1195.434,...,-131.006,0.0,285.630,458.934,-131.006,39.228,39.228,255.386,2337.9946,2.99
3584,ESI,2015,85.300,10190.200,8092.800,10190.200,645.9,0.0,2103.900,5173.600,...,0.000,0.0,-308.600,2542.300,0.000,0.000,0.000,-304.400,2944.0231,12.83
3585,NETI,2015,27.543,1485.436,1358.372,1485.436,0.0,0.0,934.531,423.841,...,0.000,0.0,-510.789,62.521,0.000,0.000,0.000,-510.789,283.7144,9.89
3586,NAO.3,2015,1.167,337.367,327.857,337.367,0.0,0.0,280.857,47.000,...,0.000,0.0,-10.844,36.372,0.000,21.922,21.922,-10.844,118.8965,5.27


#### Regressor

In [72]:
#mlp regressor 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.kernel_ridge import KernelRidge



import numpy as np

# split in x and y
x = data.drop(columns=['tic', 'fyear', 'mkvalt', 'prcc_f'])
y = data['mkvalt']

# normalize x and y
scaler = StandardScaler()
x = scaler.fit_transform(x)
# y = scaler.fit_transform(y.values.reshape(-1, 1))   

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


# MLP Regressor
mlp_regressor = MLPRegressor(
    learning_rate='adaptive',
    max_iter=1000,
    hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100),
    activation='relu',
    solver='adam',
)

mlp_regressor.fit(x_train, y_train)
score = mlp_regressor.score(x_test, y_test)
print(f'MLP Regressor Test Score: {score}')

# mlp_regressor.fit(x, y)
# score = mlp_regressor.score(x, y)
# print(f'MLP Regressor Training Score: {score}')


MLP Regressor Test Score: 0.886862427985902


#### Simulation

In [75]:
print(f'Analysing {len(data)} tickers')
mapT = {}

market_caps_2021 = market_caps_for_year(year_end)
market_caps_2020 = market_caps_for_year(year_start)

# loop through data
for i in range(len(data)):
    row = data.iloc[i]
    ticker = row['tic']
    x_i = x[i]
    # print(x_i)
    y_i = row['mkvalt']
    # mlp_regressor.predict([x_i])
    p = mlp_regressor.predict([x_i])
    diff = p[0]/y_i - 1
    # print(f'{ticker}: {p} / ${y_i} ({diff})\n')
    if diff > 0:
        mapT[ticker] = diff

# sort mapT by biggest diff
mapT = {k: v for k, v in sorted(mapT.items(), key=lambda item: item[1], reverse=True)}

#get top 30
top30 = list(mapT.keys())[:30]

count = 0
sumAll = 0
sumAllYf = 0
sumAllYfJune = 0
sumAllYfMPlus3 = 0
sumAllYfMPlus3CompletedByMkvalt = 0 # same as sumAllYfMPlus3, but uses compustat data to fill the values when they're not available
noyf = []

for tic in top30:
    vals_2021 = market_caps_2021[market_caps_2021['tic'] == tic]['price'].values
    # we also exclude some tickers here that are presenting a huge change in market cap - they don't have historical prices for 2020 for some reason, doesn't seem like a good idea to use them
    if len(vals_2021) == 0:
        continue
    m2021 = vals_2021[0] 
    m2020 = market_caps_2020[market_caps_2020['tic'] == tic]['price'].values[0] 
    change = m2021/m2020-1
    if np.isnan(change):
        change = (market_caps_2021[market_caps_2021['tic'] == tic]['mkvalt'].values[0] / market_caps_2020[market_caps_2020['tic'] == tic]['mkvalt'].values[0]) - 1
        if np.isnan(change):
            print(f'change is nan for {tic}')
            continue
    exchg = df[df['tic'] == tic]['exchg'].values[0]
    print(f'{tic}: {mapT[tic]} => {change} ({exchg})')
    yf2021 =  val_for_tic_at_year(tic, year_end)
    yf2020 = val_for_tic_at_year(tic, year_start)
    yf2021June = val_for_tic_at_year(tic, year_end, month=18)
    yf2020June = val_for_tic_at_year(tic, year_start, month=18)
    yf2021MPlus3 = val_for_tic_at_year(tic, year_end, month=only_tickers_with_fiscal_year_ending_at+3)
    yf2020MPlus3 = val_for_tic_at_year(tic, year_start, month=only_tickers_with_fiscal_year_ending_at+3)
    if not (yf2021 == None or yf2020 == None):
        changeYf = yf2021/yf2020-1
        sumAllYf += changeYf
        print(f'Yahoo Finance: {changeYf}')
    if not (yf2021June == None or yf2020June == None):
        sumAllYfJune += yf2021June/yf2020June-1
    if not (yf2021MPlus3 == None or yf2020MPlus3 == None):
        sumAllYfMPlus3 += yf2021MPlus3/yf2020MPlus3-1
        sumAllYfMPlus3CompletedByMkvalt += yf2021MPlus3/yf2020MPlus3-1
        print(f'Yahoo Finance M+3: {yf2021MPlus3/yf2020MPlus3-1}')
    else:
        sumAllYfMPlus3CompletedByMkvalt += change
        noyf.append(tic)

    sumAll += change
    count += 1
    print()

change = (sumAll*100)/count
changeYf = (sumAllYf*100)/(count - len(noyf))
changeYfJune = (sumAllYfJune)*100/(count - len(noyf))
chnageYfM3 = (sumAllYfMPlus3*100)/(count - len(noyf))
changeYfM3C = (sumAllYfMPlus3CompletedByMkvalt*100)/(count)
changeYfM3Br = ((sumAllYfMPlus3 -1*(len(noyf) + (30-count)))*100)/(count)

print(f'Average change: {change:.2f}%')
print(f'Average change yf: {changeYf:.2f}%')
print(f'Average change yf (Investing only in june after all the fillings have been made public): {changeYfJune:.2f}%')
print(f'Average change yf M+3: {chnageYfM3:.2f}%')
print(f'Average change yf M+3 (Completed by Compustat data): {changeYfM3C:.2f}%')
print(f'Average change yf M+3 (If all delisted went bankrupt): {changeYfM3Br:.2f}%')
print(f'No yf: {noyf}')
print(f'Count: {count}')
print(f'Coverage: {(count)*100/30:.2f}%')

Analysing 3588 tickers


<ipython-input-74-4af232de0991>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']
<ipython-input-74-4af232de0991>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']
<ipython-input-75-3a50d3753128>:42: RuntimeWarning: invalid value encountered in scalar divide
  change = m2021/m2020-1


ATLX: 434.34323609659606 => 4.2140215468724875 (14)

EP: 346.74984533411794 => 0.75 (12)
Yahoo Finance: 0.7333333002196434
Yahoo Finance M+3: 1.4285714437766952

OPXS: 267.6452827904013 => -0.8490196078431372 (14)
Yahoo Finance: -0.8460000068202576
Yahoo Finance M+3: -0.7083333512738251

VRS: 235.82061717440345 => 355.7839195979899 (11)
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted

BTTR: 205.36857396274357 => 52.99999999999999 (12)
Yahoo Finance: 52.99999895200625
Yahoo Finance M+3: -0.8843930628484791

WAVD: 187.57641885487493 => -0.07692307692307698 (14)
Yahoo Finance: 0.07692309808448994
Yahoo Finance M+3: 0.5714286322496385

GLBS: 169.75004347218317 => 7.4413793103448285 (14)
Yahoo Finance: 6.678571428571429
Yahoo Finance M+3: 0.8986486486486487

GBR: 128.32405

<ipython-input-75-3a50d3753128>:42: RuntimeWarning: invalid value encountered in scalar divide
  change = m2021/m2020-1



SBFM: 103.53453437379346 => 0.194071544967797 (14)


...


AEYE: 101.40440175423612 => 3.575163398692812 (14)
Yahoo Finance: 3.0
Yahoo Finance M+3: 0.17647058823529416

ENRJ: 93.20325875922813 => -0.033333333333333326 (17)
- ENRJ: No data found for this date range, symbol may be delisted
- ENRJ: No data found for this date range, symbol may be delisted
- ENRJ: No data found for this date range, symbol may be delisted
- ENRJ: No data found for this date range, symbol may be delisted
- ENRJ: No data found for this date range, symbol may be delisted
- ENRJ: No data found for this date range, symbol may be delisted

AIMD: 82.91069487929767 => 0.6875 (14)

OPHC: 74.46718298897099 => 0.016129032258064502 (14)
Yahoo Finance: 0.0027027000903734333
Yahoo Finance M+3: -0.3051282123387603

AULT: 73.45335313474908 => 0.37445331292368245 (12)


<ipython-input-75-3a50d3753128>:42: RuntimeWarning: invalid value encountered in scalar divide
  change = m2021/m2020-1


Yahoo Finance: 0.22222222222222232
Yahoo Finance M+3: 0.6499999999999999

SCWO: 70.37045446882696 => 0.08333333333333348 (14)
Yahoo Finance: 0.0833333178112905
Yahoo Finance M+3: -0.15384612739438774

ELA: 66.92051266532987 => 2.7575757575757573 (12)
Yahoo Finance: 2.9393936383603925
Yahoo Finance M+3: 1.8928571390558262

BOSC: 65.6871313429736 => 0.08717948717948731 (14)
Yahoo Finance: 0.09230772428493683

CEI: 63.15809780633626 => 0.7618705035971223 (12)
error getting hist for CEI 2016 18
Yahoo Finance: -0.821917818979921
Yahoo Finance M+3: -0.9285714308104955

BYU: 60.74415339714612 => 2.365973072215423 (14)
Yahoo Finance: 2.3636363636363638
Yahoo Finance M+3: 1.5689655172413794

CHCI: 58.040244741282095 => 0.23279227450174655 (14)
Yahoo Finance: 0.27397257925131324
Yahoo Finance M+3: 0.1918604175679559

SLNH: 56.517634622530736 => 0.5426595744680853 (14)
Yahoo Finance: 0.5425532307315486
Yahoo Finance M+3: 0.2249999301696224

POCI: 55.93187234468772 => -0.2933333333333332 (14)
Yaho

<ipython-input-75-3a50d3753128>:42: RuntimeWarning: invalid value encountered in scalar divide
  change = m2021/m2020-1


Yahoo Finance: 0.026315789473684292
Yahoo Finance M+3: -0.8898305084745762

RADA: 46.502349950257795 => 0.6338028169014085 (14)
- RADA: No data found, symbol may be delisted
- RADA: No data found, symbol may be delisted
- RADA: No data found, symbol may be delisted
- RADA: No data found, symbol may be delisted
- RADA: No data found, symbol may be delisted
- RADA: No data found, symbol may be delisted

LEU: 45.91704870268972 => 3.833333333333333 (12)
Yahoo Finance: 4.74999988711242
Yahoo Finance M+3: 0.5345268205346563

Average change: 1466.20%
Average change yf: 348.86%
Average change yf (Investing only in june after all the fillings have been made public): -3.23%
Average change yf M+3: 25.25%
Average change yf M+3 (Completed by Compustat data): 1224.30%
Average change yf M+3 (If all delisted went bankrupt): -12.33%
No yf: ['ATLX', 'VRS', 'GOVX', 'SBFM', 'ENRJ', 'AIMD', 'BOSC', 'FLUX', 'RADA']
Count: 30
Coverage: 100.00%
